In [ ]:
%load_ext autoreload
%autoreload 2

## Imports

In [ ]:
import os
import dmsbatch
from dmsbatch import create_batch_client, create_blob_client
import datetime
import logging
#logger = logging.getLogger()
#logger.setLevel(logging.ERROR)

## First create a batch client from the config file

The config file is described in the [README](../README.md)

In [ ]:
client = create_batch_client('../tests/data/dmsbatch.config')
blob_client = create_blob_client('../tests/data/dmsbatch.config')

## Application packages
To copy large files and programs it is best to zip (or targz) them and upload them as application packages

Application packages are setup separately in either azure management apis or from the web console or cli tool

These are referenced here by their name and version
e.g. DSM2, python and other programs

One extra field (last one) is the path within the zip file where the executables can be found. These are used later to setup the PATH varible

In [ ]:
app_pkgs = [('dsm2linux', '8.2.8449db2', 'DSM2-8.2.8449db2-Linux/bin')]

### Show vms available

https://docs.microsoft.com/en-us/azure/virtual-machines/fsv2-series

In [ ]:
#display(client.skus_available())

### Create or resize existing pool
If the pool doesn't exist it will create it
If the pool exists, it will resize to the second arg

In [ ]:
pool_name = 'ptmlinuxpool'
pool_start_cmds = ['printenv',
'yum install -y glibc.i686 libstdc++.i686 glibc.x86_64 libstdc++.x86_64',# --setopt=protected_multilib=false',
'yum-config-manager --add-repo https://yum.repos.intel.com/2019/setup/intel-psxe-runtime-2019.repo',
'rpm --import https://yum.repos.intel.com/2019/setup/RPM-GPG-KEY-intel-psxe-runtime-2019',
'yum install -y intel-icc-runtime-32bit intel-ifort-runtime-32bit']
client.create_pool(pool_name,
                    1,
                    app_packages=[(app,version) for app,version,_ in app_pkgs], 
                    vm_size='standard_f32s_v2', 
                    tasks_per_vm=32,
                    os_image_data=('openlogic', 'centos', '7_8'),
                    start_task_cmd=client.wrap_commands_in_shell(pool_start_cmds, ostype='linux'),
                    start_task_admin=True,
                    elevation_level='admin'
                    )

## Autoscaling Formula to use for pool
This can be added manually via the console or batch explorer in the resizing section.
```
// In this example, the pool size is adjusted based on the number of tasks in the queue. Note that both comments and line breaks are acceptable in formula strings.
numCores = 32;
// Get pending tasks for the past 15 minutes.
$samples = $ActiveTasks.GetSamplePercent(TimeInterval_Minute * 1);
// If we have fewer than 70 percent data points, we use the last sample point, otherwise we use the maximum of last sample point and the history average.
$tasks = $samples < 70 ? max(0, $ActiveTasks.GetSample(1)) : max( $ActiveTasks.GetSample(1), avg($ActiveTasks.GetSample(TimeInterval_Minute * 1)));
// If number of pending tasks is not 0, set targetVM to pending tasks, otherwise half of current dedicated nodes.
$targetVMs = $tasks > 0 ? $tasks : max(0, $TargetLowPriorityNodes / numCores);
// The pool size is capped at 20, if target VM value is more than that, set it to 20. This value should be adjusted according to your use case.
cappedPoolSize = 60;
$TargetLowPriorityNodes = max(0, min($targetVMs, cappedPoolSize));
// Set node deallocation mode - keep nodes active only until tasks finish
$NodeDeallocationOption = taskcompletion;
```

### Create job on pool or fail if it exists
Jobs are containers of tasks (things that run on nodes (machines) in the pool). If this exists, the next line will fail

In [ ]:
study_dir = 'X:/Share/xwang/DCP/neutrally_buoyant_particles/pa6k_2020'

In [ ]:
tidefile = 'X:/Share/DSM2/v821/studies_dcp_2020/pa6k_2020/output/DCP_PA6K_2020.h5'

In [ ]:
study_prefix = os.path.basename(os.path.dirname(study_dir))+'/'+os.path.basename(study_dir)
study_prefix

In [ ]:
container_name='ptmbatch'

In [ ]:
print(container_name,'%s/DCP_EX.h5'%(study_prefix),tidefile)

In [ ]:
UPLOAD=True
if UPLOAD:
    # slow - 9 mins so use max_connections > 2 (default). Using 12 which seems to be a good fit here
    blob_client.upload_file_to_container(container_name,'%s/DCP_EX.h5'%(study_prefix),tidefile,max_connections=10)
#input_tidefile = client.create_input_file_spec('ptmnbjob',blob_prefix='DCP_EX.h5',file_path='.')

In [ ]:
job_name = study_prefix.replace('/','_')
job_name

In [ ]:
copy_tidefile_task = client.create_task_copy_file_to_shared_dir(container_name,'%s/DCP_EX.h5'%(study_prefix),file_path='.',ostype='linux')
client.create_job(job_name,pool_name,prep_task=copy_tidefile_task)

In [ ]:
if UPLOAD:
    blob_client.zip_and_upload(container_name,study_prefix,study_dir,30)

In [ ]:
study_dir

### Create a task
This uses the application package as pre -set up. If not, create one https://docs.microsoft.com/en-us/azure/batch/batch-application-packages

In [ ]:
def create_ptm_single_task(task_name, run_no, task_prefix, study_prefix, envvars):
    input_file = client.create_input_file_spec(container_name,blob_prefix='%s/%s.zip'%(study_prefix,os.path.basename(study_dir)),file_path='.')
    #std_out_files = client.create_output_file_spec(
    #    '../std*.txt', output_dir_sas_url, blob_path=f'{study_prefix}/{task_prefix}/{task_name}')
    permissions = dmsbatch.commands.azureblob.BlobPermissions.WRITE
    output_dir_sas_url = blob_client.get_container_sas_url(container_name, permissions)
    output_dir = client.create_output_file_spec(
        f'{run_no}/*', output_dir_sas_url, blob_path=f'{study_prefix}/{task_prefix}/{task_name}/{run_no}')
    set_path_string = client.set_path_to_apps(app_pkgs, ostype='linux')
    zip_fname = os.path.basename(study_dir)+'.zip'
    cmd_string = client.wrap_cmd_with_app_path(
        f"""
        source /opt/intel/psxe_runtime/linux/bin/compilervars.sh ia32;
        {set_path_string};
        cd {study_prefix};
        unzip {zip_fname}; 
        rm *.zip; 
        cd studies; 
        export TIDEFILE_LOC=$AZ_BATCH_NODE_SHARED_DIR; 
        sed -i 's+./output/DCP_EX.h5+${{TIDEFILE_LOC}}/DCP_EX.h5+g' planning_ptm.inp;
        ptm planning_ptm.inp; 
        rm output/trace.out;
        mkdir -p $AZ_BATCH_TASK_WORKING_DIR/{run_no};
        mv output/* $AZ_BATCH_TASK_WORKING_DIR/{run_no};
        """, app_pkgs,ostype='linux')
    #print(cmd_string)
    ptm_task = client.create_task(f'{task_prefix}_{task_name}_{run_no}', cmd_string,
                                  resource_files=[input_file],
                                  output_files=[output_dir],
                                  env_settings=envvars)
    return ptm_task

### Create all tasks
This function looks at the insertion location file and the simulation years and months to create an array of tasks

In [ ]:
import csv
import logging
import datetime
import os
def create_tasks(study_prefix, 
               insertion_file='run_number_loc.txt',
               simulation_start_year=1923,
               simulation_end_year=2015,
               simulation_start_day=1,
               simulation_month=[1, 2, 3, 4, 5, 6],
               simulation_days=92,
               duration='1485minutes',
               delay='0day'):
    tasks = []
    study_folder, study_name = study_prefix.split('/')
    study_name,_ = study_name.split('_')
    tsnow = str(datetime.datetime.now().timestamp()).split('.')[0]
    userid = os.getlogin()
    with open(insertion_file, 'r') as input:
        for row in csv.DictReader(input):  # run#,particle#,node
            run_no = row['run#']
            particle_no = row['particle#']
            insertion_node = row['node']
            job_name_prefix = 'ptm-%s-%s-%s' % (
                study_folder[0:5], study_name, run_no)
            #
            sim_days = datetime.timedelta(days=simulation_days)
            for y in range(simulation_start_year, simulation_end_year+1):
                for m in simulation_month:
                    s_day = datetime.date(y, m, simulation_start_day)
                    e_day = s_day + sim_days
                    ptm_start_date = s_day.strftime("%d%b%Y")
                    ptm_end_date = e_day.strftime("%d%b%Y")
                    particle_insertion_row = '%s %s %s %s' % (
                        insertion_node, particle_no, delay, duration)
                    envvars = {'RUN_NO': '%s' % run_no,
                               'PTM_START_DATE': '%s' % ptm_start_date,
                               'PTM_END_DATE': '%s' % ptm_end_date,
                               'PARTICLE_INSERTION_ROW': '%s' % particle_insertion_row,
                               'DSM2_STUDY_NAME': 'DCP_%s_%sP' % (study_name, study_folder[0:1])
                               }
                    task = create_ptm_single_task(ptm_start_date, run_no, f'{userid}_{tsnow}', f'{study_prefix}', envvars)
                    tasks.append(task)
    logging.info('All done!')
    return tasks

In [ ]:
study_prefix

In [ ]:
tasks = create_tasks(study_prefix, insertion_file='d:/dev/ptm_batch/run_number_loc.txt',simulation_start_year=1923,simulation_end_year=2015)

### Next submit the task and wait 
Azure batch limits to submitting 100 tasks at a time.

In [ ]:
for i in range(0,round(len(tasks)/100)):
    client.submit_tasks(job_name,tasks[i*100:i*100+100])

## Finally resize the pool to 0 to save costs

In [ ]:
#client.resize_pool(pool_name,0)